<a href="https://colab.research.google.com/github/bazsi04/Szakdolgozat/blob/main/Algoritmusok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Nem-negatív élsúlyozás, szig. mon. csökk. legrövidebb út

In [ ]:
from heapq import heappush, heappop

def nem_neg_mon(input):
    V, adj_list, s = input
    P = []
    S = {s}
    p_v = {s: s}
    p_e = {}
    R = {s: 0}
    c = {}

    for u in adj_list:
        for (v, weight) in adj_list[u]:
            c[u,v] = weight

    h = {u: 0 for u in V}
    numedge = {u: len(adj_list[u]) for u in V}
    F = {u: sorted(adj_list[u], key=lambda x: x[1]) for u in V}

    for v, K in adj_list[s]:
        heappush(P, (K, s, v))
        p_e[s,v] = s
        if v not in R or K_total < R[t]:
            R[v] = K
            p_v[v] = (s,v)


    while P:
        K_uv, u, v = heappop(P)
        if h[v] > numedge[v]:
            continue


        for i in range(h[v], numedge[v]):
            t, K_vt = F[v][i]
            if c[v,t] >= c[u,v]:
                break

            K_total = K_uv + c[v,t]
            p_e[v,t] = (u, v)
            if t not in R or K_total < R[t]:
                R[t] = K_total
                p_v[t] = (v,t)
                heappush(P, (K_total, v, t))

            h[v] += 1

    return p_v, p_e, R


### Konzervatív élsúlyozás, szig. mon. csökk. legrövidebb út

In [ ]:
from heapq import heappush, heappop

def konz_mon(input):
    V, adj_list, s = input
    P = []
    S = {s}
    p_v = {s: s}
    p_e = {}
    R = {s: 0}
    c = {}

    for u in adj_list:
        for (v, weight) in adj_list[u]:
            c[u, v] = weight

    h = {u: 0 for u in V}
    numedge = {u: len(adj_list[u]) for u in V}
    F = {u: sorted(adj_list[u], key=lambda x: x[1]) for u in V}

    for v, cost in adj_list[s]:
        heappush(P, (cost, s, v))
        p_e[s, v] = s
        if v not in R or cost < R[v]:
            R[v] = cost
            p_v[v] = (s, v)

    while P:
        K_uv, u, v = heappop(P)
        if h[v] >= numedge[v]:
            continue

        for i in range(h[v], numedge[v]):
            t, c_vt = F[v][i]
            if c_vt >= c[u, v]:
                break

            K_total = K_uv + c_vt
            p_e[v, t] = (u, v)
            if t not in R or K_total < R[t]:
                R[t] = K_total
                p_v[t] = (v, t)
                heappush(P, (K_total, v, t))

            h[v] += 1

    return p_v, p_e, R


### Nem-negatív csúcssúlyozás minimális út

In [ ]:
def csucs_min(input):
    V, adj_list, s = input
    P = []
    S = set()
    M = set(V) - {s}
    p_v = {s: s}
    K = {s: 0}
    w = {}

    for u in adj_list:
        for v, weight in adj_list[u]:
            w[v] = weight

    heappush(P, (w.get(s, 0), s))
    while P:
        K_u, u = heappop(P)
        if u in S:
            continue
        S.add(u)

        for v, _ in adj_list.get(u, []):
            if v in M:
                K[v] = K[u] + w.get(v, 0)
                p_v[v] = p_v[u]
                heappush(P, (K[v], v))
                M.remove(v)

    return p_v, K

### Legfeljebb k élű minimális út

In [ ]:
def k_el_min(input):
    V, adj_list, s, k = input
    K = {}
    p = {}
    c = {}

    for u in adj_list:
        for (v, weight) in adj_list[u]:
            c[(u, v)] = weight


    for u in V:
        K[(0, u)] = float('inf')
    K[(0, s)] = 0
    p[(0, s)] = s

    for j in range(1, k + 1):
        for u in V:
            K[(j, u)] = float('inf')
            p[(j, u)] = p.get((j - 1, u), None)

        for u in adj_list:
            for v, cost in adj_list[u]:
                prev = K.get((j - 1, u), float('inf'))
                if K[(j, v)] > prev + cost:
                    K[(j, v)] = prev + cost
                    p[(j, v)] = u

    return K, p

### k-hosszú út

In [ ]:
def coloring(V, adj_list, k, seed = None):
    if seed is not None:
        random.seed(seed)

    node_colors = {v: random.randint(0, k-1) for v in V}


    edge_colors = {}
    for u in adj_list:
        for v, _ in adj_list[u]:
            edge_colors[(u, v)] = node_colors[v]

    return node_colors, edge_colors

In [ ]:
from itertools import combinations
import random

def colorcoding(input, seed = None):
    V, adj_list, s, k = input
    from collections import defaultdict
    f = {}
    p = {}
    colors = set(range(0,k))
    _, edge_colors = coloring(V, adj_list, k, seed)
    #print("colors: ",edge_colors)
    w = {}
    for u in adj_list:
        for (v, weight) in adj_list[u]:
            w[u, v] = weight

    E = [(u, v) for u in adj_list for (v,_) in adj_list[u]]

    rev_adj_list = {}
    # Színek részhalmazai elemszám szerint növekvő sorrendben
    P = [frozenset(comb) for i in range(1, k+1) for comb in combinations(colors, i)]


    for p_set in P:
        for (u, v) in E:
            f[p_set, (u, v)] = False


            if u == s and edge_colors[(u, v)] in p_set:
                f[p_set, (u, v)] = True
                p[p_set, (u, v)] = s


    for p_set in P[k:]:
        for (u, v) in E:
            f[p_set, (u, v)] = False
            if edge_colors[(u, v)] in p_set:
                for (t, uu) in E:
                    if uu == u and f.get((p_set - {edge_colors[(u, v)]}, (t, uu)), True) and w[(t, uu)] < w[(u, v)] and v != s:
                        f[p_set, (u, v)] = True
                        p[p_set, (u, v)] = (t, u)

    for p_set in P[k:]:
        for (u, v) in E:
            f[p_set, (u, v)] = False
            if edge_colors[(u, v)] in p_set:
                for (t, uu) in E:
                    if uu == u and f.get((p_set - {edge_colors[(u, v)]}, (t, uu)), True) and w[(t, uu)] < w[(u, v)] and v != s:
                        f[p_set, (u, v)] = True
                        p[p_set, (u, v)] = (t, u)


    f_v = {}
    p_v = {}
    for (u, v) in E:
        if f.get((P[-1], (u, v)), True):
            f_v[v] = True
            p_v[v] = (u, v)

    return f_v, p_v, f, p

In [ ]:
from itertools import combinations
import random

def colorcoding_fast(input, seed = None):
    V, adj_list, s, k = input
    from collections import defaultdict
    f = {}
    p = {}
    colors = set(range(0,k))
    _, edge_colors = coloring(V, adj_list, k, seed)
    #print("colors: ",edge_colors)
    """
    w = {}
    for u in adj_list:
        for (v, weight) in adj_list[u]:
            w[u, v] = weight

    E = [(u, v) for u in adj_list for (v,_) in adj_list[u]]
    """
    rev_lst = {u: [] for u in adj_list}

    for u in adj_list:
        for v, weight in adj_list[u]:
            rev_lst[v].append((u, weight))

    # Színek részhalmazai elemszám szerint növekvő sorrendben
    P = [frozenset(comb) for i in range(1, k+1) for comb in combinations(colors, i)]


    for p_set in P:
        for u in adj_list:
            for v, _ in adj_list[u]:
                f[p_set, (u, v)] = False


            if u == s and edge_colors[(u, v)] in p_set:
                f[p_set, (u, v)] = True
                p[p_set, (u, v)] = s

    """
    for p_set in P[k:]:
        for (u, v) in E:
            f[p_set, (u, v)] = False
            if edge_colors[(u, v)] in p_set:
                for (t, uu) in E:
                    if uu == u and f.get((p_set - {edge_colors[(u, v)]}, (t, uu)), True) and w[(t, uu)] < w[(u, v)] and v != s:
                        f[p_set, (u, v)] = True
                        p[p_set, (u, v)] = (t, u)
    """
    for p_set in P[k:]:
        for u in adj_list:
            for (v, w) in adj_list[u]:
                f[p_set, (u, v)] = False
                if edge_colors[(u, v)] in p_set:
                    for (t, ww) in rev_lst[u]:
                        if f.get((p_set - {edge_colors[(u, v)]}, (t, u)), True) and ww < w and v != s:
                            f[p_set, (u, v)] = True
                            p[p_set, (u, v)] = (t, u)


    f_v = {}
    p_v = {}
    """
    for (u, v) in E:
        if f.get((P[-1], (u, v)), True):
            f_v[v] = True
            p_v[v] = (u, v)
    """
    for u in adj_list:
        for v, _ in adj_list[u]:
            if f.get((P[-1], (u, v)), True):
                f_v[v] = True
                p_v[v] = (u, v)

    return f_v, p_v, f, p